####1. Sharing data for External Delta Table 

In [0]:
%fs ls abfss://dbfs-container@adlssmiloeabronzedev001.dfs.core.windows.net

In [0]:
%python
base_dir="abfss://dbfs-container@adlssmiloeabronzedev001.dfs.core.windows.net"
# Read the json file into a DataFrame
df = spark.read.json(f'{base_dir}/flight-time.json')

create credential for the external location, click on catalog-> external data-> credentials-> create credetnails "credential_el_bronze". while creating credntial give credential name and access connector resource id"

#### Create external location 

In [0]:
%sql
CREATE EXTERNAL LOCATION IF NOT EXISTS external_data01
URL 'abfss://dbfs-container02@adlssmiloeabronzedev001.dfs.core.windows.net'
WITH (CREDENTIAL `credential_el_bronze`)


#####2. Save data in delta format

In [0]:

(df.coalesce(1)
    .write
    .format("delta")
    .mode("overwrite")
    .save("abfss://dbfs-container02@adlssmiloeabronzedev001.dfs.core.windows.net/flight-time"))

Create unity catalog table using the delt table

In [0]:
%sql
USE CATALOG dev;
USE SCHEMA demo_db;
CREATE TABLE flight_time_external_table
USING delta
AS SELECT *
FROM delta.`abfss://dbfs-container02@adlssmiloeabronzedev001.dfs.core.windows.net/flight-time`

### the other way to create unity catalog table by mentioning the explicit ddl
%sql
CREATE TABLE IF NOT EXISTS dev.demo_db.flight_time_ext_tbl (
    FL_DATE DATE, 
    OP_CARRIER STRING, 
    OP_CARRIER_FL_NUM INT, 
    ORIGIN STRING, 
    ORIGIN_CITY_NAME STRING, 
    DEST STRING, 
    DEST_CITY_NAME STRING, 
    CRS_DEP_TIME INT, 
    DEP_TIME INT, 
    WHEELS_ON INT, 
    TAXI_IN INT, 
    CRS_ARR_TIME INT, 
    ARR_TIME INT, 
    CANCELLED STRING, 
    DISTANCE INT
) USING DELTA
LOCATION "abfss://dbfs-container@prashantsa.dfs.core.windows.net/external/flight-time"

##### 2. Reading Delta Table

In [0]:
%sql
select * from dev.demo_db.flight_time_external_table limit 5

In [0]:
df=spark.read.format("delta").table("dev.demo_db.flight_time_external_table").limit(5)
display(df)

In [0]:
df=spark.read.format("delta").load("abfss://dbfs-container02@adlssmiloeabronzedev001.dfs.core.windows.net/flight-time").limit(5)
display(df)

####Delta Table Operations

In [0]:
%sql
SELECT * FROM dev.demo_db.people;

####1. Delete one record from the above table using Spark SQL

In [0]:
%sql
delete from dev.demo_db.people where fname = "M David"

In [0]:
%sql
SELECT * FROM dev.demo_db.people;

###2. Delete one record from the above table using API

In [0]:
from delta import DeltaTable

people_dt = DeltaTable.forName(spark, "dev.demo_db.people")
people_dt.delete("fname = 'abdul'")

In [0]:
%sql
SELECT * FROM dev.demo_db.people;

###3. Update one record in the delta table using API

In [0]:
import pyspark.sql.functions as f
people_dt.update(
  condition = "dob = '1975-05-25'",
  set = { "fname": f.initcap("fname"), "lname":  f.initcap("lname") }
)

In [0]:
%sql
SELECT * FROM dev.demo_db.people;

###4. Merge the given dataframe into the delta table

In [0]:
%fs ls /mnt/u01

In [0]:
source_df = spark.read.format("json").load("/mnt/u01/people.json")
display(source_df)


In [0]:
(people_dt.alias("tgt")
    .merge(source_df.alias("src"), "src.id=tgt.id")
    .whenMatchedDelete(condition="tgt.fname='Kailash' and tgt.lname='Patil'")
    .whenMatchedUpdate(condition="tgt.id = 101", set = {"tgt.dob": "src.dob"})
    .whenMatchedUpdate(set = {"tgt.id": "src.id", "tgt.fname":"src.fname", "tgt.lname":"src.lname", "tgt.dob":"src.dob"})
    .whenNotMatchedInsert(values = {"tgt.id": "src.id", "tgt.fname":"src.fname", "tgt.lname":"src.lname", "tgt.dob":"src.dob"})
    .execute()
)

In [0]:
%sql
SELECT * FROM dev.demo_db.people;